In [6]:
import json
import numpy as np
from scipy.special import softmax
from tqdm import tqdm
from pprint import pprint
from sklearn.cluster import KMeans
from munkres import Munkres

from label_processor import CStdLib

In [7]:
with open('result_logits.json', encoding='utf-8') as f:
    logits_json = json.load(f)

In [8]:
embed = CStdLib(single=False)

In [9]:
pbar = tqdm(logits_json.items())
munkres = Munkres()
i = 0
for key, prod_dict in pbar:
    optional_tags = prod_dict['optional_tags']
    num_class = len(optional_tags)
    imgs_tags = prod_dict['imgs_tags']
    imgs_logits = []
    for i in range(len(imgs_tags)):
        imgs_logits.append(np.fromstring(imgs_tags[i][key + '_{}.jpg'.format(i)], dtype=np.float32, sep=' '))
    imgs_logits = np.array(imgs_logits)
    
    # 聚类
    cluster = KMeans(n_clusters=num_class, random_state=42).fit(imgs_logits)
    labels = cluster.labels_
    
    # 计算匹配度
    match_score = np.zeros((num_class, num_class))
    # match_score[i, j] 表示 kmeans 分类结果为 i 和 optional_tags[j] 的匹配度
    embeded_tags = [embed(tag) for tag in optional_tags]
    not_matched = False
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        logits = imgs_logits[id_i]
        for j in range(num_class):
            logits_ids = embeded_tags[j]
            if len(logits_ids) > 0:
                logits_j = logits[:, logits_ids]
                match_score[i, j] = np.mean(logits_j)
            else:
                not_matched = True
                match_score[i, j] = 1
    
    # 对 scores 按列归一化
    match_score = (np.abs(match_score) +  match_score) / 2. + 1e-7 # relu
    match_score_norm = match_score / np.linalg.norm(match_score, axis=0, ord=1)
    # match_score_norm = softmax(match_score, axis=0)
    
    # 使用匈牙利算法计算最优匹配
    risk_matrix = 1. - match_score_norm
    munkres_result = munkres.compute(risk_matrix)
    
    # 终于可以写入 json 了
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        label = optional_tags[munkres_result[i][1]]
        for j in id_i:
            logits_json[key]['imgs_tags'][j][key + '_{}.jpg'.format(j)] = label
    if not_matched:
        print(key, '\n', match_score, '\n', match_score_norm)
        pprint(logits_json[key])

  5%|▌         | 279/5331 [00:03<01:09, 73.08it/s] 

624952669722 
 [[3.42698036 1.0000001 ]
 [5.39661608 1.0000001 ]] 
 [[0.38838816 0.5       ]
 [0.61161184 0.5       ]]
{'imgs_tags': [{'624952669722_0.jpg': '珊瑚色组合'},
               {'624952669722_1.jpg': '珊瑚色组合'},
               {'624952669722_2.jpg': '橄榄色组合'},
               {'624952669722_3.jpg': '橄榄色组合'},
               {'624952669722_4.jpg': '橄榄色组合'}],
 'optional_tags': ['珊瑚色组合', '橄榄色组合']}


  7%|▋         | 348/5331 [00:03<00:47, 105.65it/s]

614043736149 
 [[ 5.664219    6.80937682  3.0382763   1.0000001   5.91158305]
 [ 3.8386942   3.8388711  11.31113253  1.0000001   3.06997309]
 [ 3.59171877 11.78814326  4.34701214  1.0000001   2.77561102]
 [12.11357317  3.26983796  3.10478507  1.0000001   2.78206406]
 [ 4.20342074  3.02493964  2.04442153  1.0000001  12.64851389]] 
 [[0.19258436 0.23700313 0.1274144  0.2        0.21743558]
 [0.13051622 0.13361347 0.47434828 0.2        0.11291753]
 [0.12211901 0.41029111 0.18229808 0.2        0.10209052]
 [0.41186343 0.11380804 0.13020354 0.2        0.10232787]
 [0.14291698 0.10528425 0.0857357  0.2        0.4652285 ]]
{'imgs_tags': [{'614043736149_0.jpg': '条纹'},
               {'614043736149_1.jpg': '桔色'},
               {'614043736149_2.jpg': '红色'},
               {'614043736149_3.jpg': '桔色'},
               {'614043736149_4.jpg': '红色'},
               {'614043736149_5.jpg': '黑色'},
               {'614043736149_6.jpg': '条纹'},
               {'614043736149_7.jpg': '黑色'},
               {

 15%|█▌        | 813/5331 [00:08<00:50, 88.91it/s] 

...

 85%|████████▍ | 4529/5331 [00:42<00:06, 118.79it/s]

577541680180 
 [[14.86534319  1.0000001   6.96493731  2.01583348]
 [ 2.39070569  1.0000001   2.37925969 11.59864817]
 [ 4.04465399  1.0000001  11.47464762  3.51491008]
 [ 3.49112401  1.0000001   4.32417927  9.89799605]] 
 [[0.59960661 0.25       0.27701271 0.07458484]
 [0.0964312  0.25       0.09462902 0.42914425]
 [0.16314465 0.25       0.456375   0.13004994]
 [0.14081754 0.25       0.17198326 0.36622097]]
{'imgs_tags': [{'577541680180_0.jpg': '红色'},
               {'577541680180_1.jpg': '红色'},
               {'577541680180_2.jpg': '蓝色'},
               {'577541680180_3.jpg': '蓝色'},
               {'577541680180_4.jpg': '橙色'},
               {'577541680180_5.jpg': '深色'},
               {'577541680180_6.jpg': '橙色'},
               {'577541680180_7.jpg': '蓝色'},
               {'577541680180_8.jpg': '蓝色'}],
 'optional_tags': ['橙色', '深色', '红色', '蓝色']}


 90%|████████▉ | 4781/5331 [00:44<00:04, 124.01it/s]

612126042525 
 [[ 1.0000001  13.50281916]
 [ 1.0000001  12.25867472]] 
 [[0.5        0.52414737]
 [0.5        0.47585263]]
{'imgs_tags': [{'612126042525_0.jpg': '957黑色'},
               {'612126042525_1.jpg': '957黑色'},
               {'612126042525_2.jpg': '957黑色'},
               {'612126042525_3.jpg': '6802格子'},
               {'612126042525_4.jpg': '957黑色'},
               {'612126042525_5.jpg': '957黑色'},
               {'612126042525_6.jpg': '957黑色'}],
 'optional_tags': ['6802格子', '957黑色']}


100%|██████████| 5331/5331 [00:49<00:00, 107.53it/s]


In [10]:
with open('./result_labels.json', 'w') as f:
    json.dump(logits_json, f, indent=4, ensure_ascii=False)